In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
#dealing with mnist, 10 output channels requireed minm
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #28x28 (initially)
        #target rf ~ 28

        self.conv1 = nn.Sequential(  #26, rf = 3
            nn.Conv2d(1, 16, 3, padding=1, stride = 1, bias = False), #input -?(28) OUtput? = (28) RF = (3)
            nn.ReLU(),

        )

        ### TRANSITION BLOCK
        self.tb = nn.Sequential( #14
            nn.Conv2d(16, 10, 1, padding = 0, stride = 1, bias = False) #size = 11 , rf = 8
        )
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv4 = nn.Sequential(
            nn.Conv2d(10, 16, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential( #size 3
            nn.Conv2d(16, 16, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
        )
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv6 = nn.Sequential( #size 1
            nn.Conv2d(16, 10, 1, padding = 0 , stride = 1, bias = False)
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding = 1 , stride = 1, bias = False),
            nn.ReLU(),
        )

        self.conv8 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
        )

        self.conv9 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding = 0 , stride = 1, bias = False)
        )

        #size 1, rf = 29

    def forward(self, x):

        x = self.conv1(x)
        x = self.tb(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool2(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x) #2 10 1 1
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

#CB2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 10, 28, 28]             160
         MaxPool2d-4           [-1, 10, 14, 14]               0
            Conv2d-5           [-1, 16, 12, 12]           1,440
              ReLU-6           [-1, 16, 12, 12]               0
            Conv2d-7           [-1, 16, 10, 10]           2,304
              ReLU-8           [-1, 16, 10, 10]               0
         MaxPool2d-9             [-1, 16, 5, 5]               0
           Conv2d-10             [-1, 10, 5, 5]             160
           Conv2d-11             [-1, 10, 5, 5]             900
             ReLU-12             [-1, 10, 5, 5]               0
           Conv2d-13             [-1, 10, 3, 3]             900
             ReLU-14             [-1, 1

<ipython-input-7-930aae46743c>:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
from tqdm import tqdm
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        y_pred = model(data)
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("EPOCH: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH:  1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-7-930aae46743c>:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.11400545388460159 Batch_id=468 Accuracy=42.26: 100%|██████████| 469/469 [00:19<00:00, 24.08it/s]



Test set: Average loss: 0.2127, Accuracy: 9358/10000 (94%)

EPOCH:  2


Loss=0.08562730997800827 Batch_id=468 Accuracy=95.39: 100%|██████████| 469/469 [00:18<00:00, 25.62it/s]



Test set: Average loss: 0.0992, Accuracy: 9696/10000 (97%)

EPOCH:  3


...


Test set: Average loss: 0.0390, Accuracy: 9878/10000 (99%)

EPOCH:  14


Loss=0.011376998387277126 Batch_id=468 Accuracy=99.15: 100%|██████████| 469/469 [00:18<00:00, 25.42it/s]



Test set: Average loss: 0.0360, Accuracy: 9886/10000 (99%)

EPOCH:  15


Loss=0.013470571488142014 Batch_id=468 Accuracy=99.24: 100%|██████████| 469/469 [00:19<00:00, 24.25it/s]



Test set: Average loss: 0.0359, Accuracy: 9881/10000 (99%)

